<a href="https://colab.research.google.com/github/hucarlos08/GEE-CIMAT/blob/main/TasseledTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tasseled Cap Transformation

## Background and Concept

The Tasseled Cap Transformation was originally developed by Kauth and Thomas (1976) based on their observations of agricultural land cover data from early Landsat sensors. When plotting pixel values from the Near-Infrared (NIR) and Red spectral bands, they noticed that data points for vegetation tended to form a characteristic shape – resembling a "cap" – as the crop cycle progressed from bare soil, through green growth, to senescence (yellowing), and finally harvest.

The TCT is essentially a **linear transformation** – specifically, a **rotation** of the spectral axes in the multi-dimensional space defined by the satellite's bands. The goal is to transform the original correlated band data into a new set of uncorrelated components (axes) that are directly associated with physical characteristics of the land surface.

## The TCT Components (Axes)

The transformation aims to maximize the spectral separation between key ground features. For sensors like Landsat TM, ETM+, and OLI, the first three components typically capture the vast majority of spectral information and have well-defined physical interpretations:

1.  **Brightness:** This first component is aligned with the principal direction of soil reflectance variation (the "soil line"). It's primarily sensitive to overall scene reflectance, influenced by factors like soil type, soil moisture (though inversely), and atmospheric conditions. Think of it as a weighted sum of all reflectances representing overall brightness.
2.  **Greenness:** Designed to be orthogonal (perpendicular) to Brightness, this component contrasts the NIR band strongly against the visible bands (especially Red). It is highly sensitive to the amount of healthy green vegetation, directly related to chlorophyll content and leaf structure. High greenness values indicate vigorous vegetation.
3.  **Wetness:** Orthogonal to both Brightness and Greenness, this third component is sensitive to moisture content in both soil and vegetation canopies. It also relates to vegetation structure and senescence. Water bodies typically exhibit high wetness values. In the original Kauth & Thomas formulation for wheat, this axis also captured the "yellow stuff" or ripening stage.

Higher-order components (Fourth, Fifth, Sixth, etc.) capture remaining variability, often related to atmospheric effects like haze, or sensor noise, and are generally less used for thematic mapping.

## Mathematical Formulation

The transformation takes the original vector of pixel reflectance values ($p_0$) across multiple bands and rotates it into a new vector ($p_1$) using a predefined transformation matrix ($R^T$, the transpose of an orthonormal basis matrix $R$):

$$ p_1 = R^T p_0 $$

*   $p_0$: The original pixel vector (e.g., reflectance values for Bands 1-5 and 7 for Landsat TM).
*   $R^T$: The Tasseled Cap coefficient matrix (transpose). **Crucially, this matrix is specific to the sensor** (e.g., Landsat 5 TM, Landsat 7 ETM+, Landsat 8 OLI, Sentinel-2 MSI) because each sensor has different band characteristics (wavelengths, spectral response functions).
*   $p_1$: The resulting pixel vector containing the Brightness, Greenness, Wetness, etc., component values.

## Importance of Surface Reflectance

While the coefficients were originally derived from various data types, **it is generally best practice to apply Tasseled Cap transformations to Surface Reflectance (SR)** data rather than Top-of-Atmosphere (TOA) reflectance. SR data has been corrected for atmospheric effects (scattering, absorption), providing a more stable and physically meaningful representation of the surface conditions. Using SR ensures that the TCT components (especially Brightness and Wetness) are less confounded by atmospheric variability and are more directly related to ground conditions across different images and dates. Coefficients specifically derived for SR data should be used (e.g., Baig et al., 2014 for Landsat 8 SR).

## Implementation in Google Earth Engine

In GEE, we implement TCT using array transformations. The process involves:

1.  Defining the sensor-specific TCT coefficient matrix ($R^T$) as an `ee.Array`.
2.  Selecting the appropriate bands from the input image (usually SR).
3.  Converting the selected bands into an `ee.Array` image (typically a 2D array per pixel, e.g., 6 rows x 1 column).
4.  Performing matrix multiplication between the coefficient array and the image array using `matrixMultiply()`.
5.  Projecting and flattening the result back into a multi-band image where each band represents a TCT component (Brightness, Greenness, Wetness, etc.) using `arrayProject()` and `arrayFlatten()`.

In [ ]:
import ee
import folium
import math # Used for isnan check if needed, though less likely with GEE server-side

# Helper function to add GEE tile layers to a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  """Adds a Google Earth Engine tile layer to a Folium map."""
  # Handle potential null or invalid image objects gracefully
  if ee_image_object is None:
      print(f"Warning: Skipping layer '{name}' because the Earth Engine Image object is null.")
      return self # Return the map object unchanged

  try:
      map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
      folium.raster_layers.TileLayer(
          tiles=map_id_dict['tile_fetcher'].url_format,
          attr='Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
          name=name,
          overlay=True,
          control=True
      ).add_to(self)
      print(f"Layer '{name}' added successfully.")
  except Exception as e:
      print(f"Error adding layer '{name}': {e}")
      # Optionally, try to get more info if it's an EEException
      # if isinstance(e, ee.ee_exception.EEException):
      #     print(f"EEException details: {e}")
  return self # Return the map object even if adding layer failed

# Add the helper function as a method to the folium.Map class
folium.Map.add_ee_layer = add_ee_layer

print("Libraries imported and helper function defined.")

# -----------------------------------------------------------------------------
# Authenticate and Initialize Google Earth Engine
# -----------------------------------------------------------------------------
try:
    ee.Initialize()
    print("Google Earth Engine initialized successfully.")
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    print("Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project='ee-cimat')
    print("Google Earth Engine authenticated and initialized successfully.")

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------

# 1. Define Area of Interest (AOI)
# Using the coordinates for Odessa, Washington, USA from the source text.
aoi = ee.Geometry.Point([-118.7436019417829, 47.18135755009023])
print(f"AOI defined for Odessa, WA: {aoi.getInfo()}") # getInfo() to see coords

# 2. Define Time Period
# Using the timeframe from the source text.
start_date = '2008-06-01'
end_date = '2008-09-01'
print(f"Time period set from {start_date} to {end_date}.")

# 3. Select Landsat Collection and Bands
# We choose Landsat 5 Collection 2 Surface Reflectance (SR).
# Collection 2 uses '_L2' for Level-2 (SR) products.
landsat_collection_id = 'LANDSAT/LT05/C02/T1_L2'
# Bands needed for TCT with Landsat 5/7 (TM/ETM+): Blue, Green, Red, NIR, SWIR1, SWIR2
# These correspond to SR_B1, SR_B2, SR_B3, SR_B4, SR_B5, SR_B7 in Collection 2 SR.
tct_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
# Bands for True Color visualization (Red, Green, Blue)
rgb_bands = ['SR_B3', 'SR_B2', 'SR_B1'] # Note the order for RGB
print(f"Using Landsat Collection: {landsat_collection_id}")
print(f"Bands for TCT: {tct_bands}")
print(f"Bands for RGB: {rgb_bands}")

# -----------------------------------------------------------------------------
# Tasseled Cap Coefficients for Landsat 5 TM Surface Reflectance
# -----------------------------------------------------------------------------
# IMPORTANT: Coefficients are sensor-specific and depend on whether TOA or SR
# is used. These are commonly cited coefficients for Landsat TM SR,
# derived from or similar to Crist (1985) but adapted/validated for SR.
# Ensure these match the bands selected in tct_bands order:
# [Blue, Green, Red, NIR, SWIR1, SWIR2] -> [SR_B1, SR_B2, SR_B3, SR_B4, SR_B5, SR_B7]

# The matrix RT (transpose of R) where each row corresponds to a component
tasseled_cap_coefficients = ee.Array([
  # Brightness
  [0.2043, 0.4158, 0.5524, 0.5741, 0.3124, 0.2303],
  # Greenness
  [-0.1603, -0.2819, -0.4934, 0.7940, -0.0002, -0.1446],
  # Wetness
  [0.0315, 0.2021, 0.3102, 0.1594, -0.6806, -0.6109],
  # Fourth (Haze/Other) - included for completeness, often less used
  [0.8172, -0.0010, -0.0600, 0.0169, -0.1794, -0.5383] # Signs might vary based on source/convention
  # Add Fifth and Sixth rows if needed/available
])
print("Tasseled Cap coefficients for Landsat 5 TM SR defined.")

# Component names corresponding to the rows in the coefficient matrix
component_names = ['brightness', 'greenness', 'wetness', 'fourth']
print(f"Component names: {component_names}")

# -----------------------------------------------------------------------------
# Data Loading and Preprocessing
# -----------------------------------------------------------------------------

# Function to apply scaling factors for Landsat Collection 2 SR data
# Output is reflectance values between 0 and 1 (approximately)
def apply_scale_factors(image):
  # Apply scaling factors to optical bands
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # Apply scaling factor to thermal band (if selected, e.g., ST_B6)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  # Add scaled bands back to the image, overwriting the original bands
  return image.addBands(optical_bands, None, True)\
              .addBands(thermal_bands, None, True)

# Load, filter, and preprocess the image collection
landsat_collection = (ee.ImageCollection(landsat_collection_id)
                      .filterBounds(aoi)
                      .filterDate(start_date, end_date)
                      .filter(ee.Filter.lt('CLOUD_COVER', 20)) # Filter for low cloud cover
                      .map(apply_scale_factors) # Apply scaling factors
                      .sort('CLOUD_COVER')) # Sort by cloud cover, ascending

# Get the least cloudy image
image_l5_sr = landsat_collection.first()

# Check if an image was found
if image_l5_sr.bandNames().size().getInfo() == 0: # Check if the image has bands
    print("ERROR: No suitable Landsat image found for the specified criteria.")
    print("Try adjusting the date range, AOI, or cloud cover filter.")
    # Stop execution or handle this case as needed
    # exit() # Uncomment to stop script if needed
    tassel_cap_image = None # Set TCT image to None if source is missing
else:
    print("Least cloudy Landsat 5 SR image acquired and scaled.")
    # Optional: Print image metadata
    # print("Image ID:", image_l5_sr.id().getInfo())
    # print("Acquisition Date:", ee.Date(image_l5_sr.get('system:time_start')).format('YYYY-MM-dd').getInfo())
    # print("Cloud Cover:", image_l5_sr.get('CLOUD_COVER').getInfo())

    # -----------------------------------------------------------------------------
    # Tasseled Cap Transformation Calculation
    # -----------------------------------------------------------------------------

    # 1. Select the required bands (already scaled)
    image_bands = image_l5_sr.select(tct_bands)

    # 2. Convert the multiband image to an array image.
    # First, create a 1D array image (list of values per pixel)
    array_image_1d = image_bands.toArray()

    # 3. Convert the 1D array image to a 2D array image (matrix per pixel)
    # We need a column vector (e.g., 6 rows x 1 column) to match p0 in the formula p1 = RT * p0.
    # The second argument '1' in toArray specifies the axis along which to stack bands.
    array_image_2d = array_image_1d.toArray(1)

    # 4. Perform matrix multiplication: RT * p0
    # Convert the coefficient ee.Array to an ee.Image for the multiplication.
    coefficient_image = ee.Image(tasseled_cap_coefficients)
    # Multiply the coefficient matrix image by the 2D band array image.
    tct_components_array = coefficient_image.matrixMultiply(array_image_2d)

    # 5. Project and Flatten the result
    # The result of matrixMultiply is a 2D array (e.g., 4x1).
    # We want to extract the component values. arrayProject([0]) selects the first axis (the components).
    tct_components_projected = tct_components_array.arrayProject([0])

    # Convert the array image back to a multiband image.
    # arrayFlatten takes the projected array and creates bands based on the provided list of names.
    tassel_cap_image = tct_components_projected.arrayFlatten([component_names])

    print("Tasseled Cap Transformation calculation complete.")

# -----------------------------------------------------------------------------
# Visualization
# -----------------------------------------------------------------------------

# 1. Define Visualization Parameters for RGB
# Using SR bands, reflectance values are typically 0-0.3 for good contrast in vegetated areas.
rgb_vis_params = {
  'bands': rgb_bands, # Correct order: SR_B3, SR_B2, SR_B1 for Red, Green, Blue
  'min': 0.0,
  'max': 0.3, # Adjust max based on image brightness if needed (e.g., 0.25 or 0.35)
  'gamma': 1.4 # Adjust gamma for brightness/contrast
}
print("RGB visualization parameters defined.")

# 2. Define Visualization Parameters for TCT
# Displaying Brightness (Red), Greenness (Green), Wetness (Blue)
# Min/max values for TCT components can vary, these are typical starting points for SR.
# You might need to inspect the image values ('Inspector' tab in GEE Code Editor)
# or calculate statistics to refine these ranges for optimal contrast.
tct_vis_params = {
  'bands': ['brightness', 'greenness', 'wetness'], # Map components to RGB
  'min': [-0.1, -0.1, -0.1], # Min value for B, G, W respectively
  'max': [ 0.5,  0.2,  0.1]  # Max value for B, G, W respectively (Adjust these!)
  # Example: Increase Brightness max if scene is very bright, adjust Greenness/Wetness based on vegetation/moisture range
}
print("TCT visualization parameters defined.")

# 3. Create a Folium Map
map_center = aoi.coordinates().getInfo()[::-1] # Reverse coords for Folium [lat, lon]
zoom_level = 11 # Zoom level similar to source text example

map_tct = folium.Map(location=map_center, zoom_start=zoom_level)
print(f"Folium map initialized, centered at {map_center} with zoom level {zoom_level}.")

# 4. Add Layers to the Map
# Add the True Color SR image first as a base reference
map_tct = add_ee_layer(map_tct, image_l5_sr, rgb_vis_params, 'Landsat 5 SR (True Color)')

# Add the Tasseled Cap image (if calculation was successful)
map_tct = add_ee_layer(map_tct, tassel_cap_image, tct_vis_params, 'Tasseled Cap (B,G,W -> R,G,B)')

# 5. Add Layer Control
map_tct.add_child(folium.LayerControl())
print("Added Layer Control to the map.")

# 6. Display the Map
display(map_tct)

Libraries imported and helper function defined.
Earth Engine initialization failed: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.
Attempting authentication...
Google Earth Engine authenticated and initialized successfully.
AOI defined for Odessa, WA: {'type': 'Point', 'coordinates': [-118.7436019417829, 47.18135755009023]}
Time period set from 2008-06-01 to 2008-09-01.
Using Landsat Collection: LANDSAT/LT05/C02/T1_L2
Bands for TCT: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
Bands for RGB: ['SR_B3', 'SR_B2', 'SR_B1']
Tasseled Cap coefficients for Landsat 5 TM SR defined.
Component names: ['brightness', 'greenness', 'wetness', 'fourth']
Least cloudy Landsat 5 SR image acquired and scaled.
Tasseled Cap Transformation calculation complete.
RGB visualization parameters defined.
TCT visualization parameters defined.
Folium map initialized, centered at [47.18135755009

--- Script finished ---
